This NoteBook will be used for IBM Data Science Capstone Project
Let us first import the libraries

In [5]:
import requests
import pandas as pd
print("Libraries imported")

Libraries imported


Scrap the Neighbourhood and Population Data of Toronto.

In [6]:
link = "https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/2fd345a6-4107-49c1-b439-5ba0abe011d7"
df = pd.read_excel(link, sheet_name=1, header=0)
df

,HoodID,Neighbourhood,Pop2016,Pop2011,PopChg11t16
0,1,West Humber-Clairville,33312,34100,-0.023109
1,2,Mount Olive-Silverstone-Jamestown,32954,32788,0.005063
2,3,Thistletown-Beaumond Heights,10360,10138,0.021898
3,4,Rexdale-Kipling,10529,10488,0.003909
4,5,Elms-Old Rexdale,9456,9550,-0.009843
...,...,...,...,...,...
135,136,West Hill,27392,26547,0.031830
136,137,Woburn,53485,53350,0.002530
137,138,Eglinton East,22776,22829,-0.002322
138,139,Scarborough Village,16724,16609,0.006924


Find the latitude and longitude data of each neighbourhood.

In [6]:
Toronto_Data = pd.DataFrame(columns=["Neighbourhood", "Latitude", "Longitude","Population (As on 2016)"])


for i in range(df.shape[0]):
    neighbourhood = df.at[i, "Neighbourhood"]
    population = df.at[i, "Pop2016"]
    URL = "https://geocode.search.hereapi.com/v1/geocode"
    location = neighbourhood
    api_key = 'jT04cMJijh0Vvl7c3rDcVGXPXAHjQipPNJMyugBhOco' # Acquire from developer.here.com
    PARAMS = {'apikey':api_key,'q':location} 

    # sending get request and saving the response as response object 
    r = requests.get(url = URL, params = PARAMS) 
    data = r.json()

    latitude = data['items'][0]['position']['lat']
    longitude = data['items'][0]['position']['lng']
    
    
    Toronto_Data = Toronto_Data.append({"Neighbourhood":neighbourhood, "Latitude":latitude, "Longitude":longitude, "Population (As on 2016)" :population}, ignore_index=True)

Toronto_Data

,Neighbourhood,Latitude,Longitude,Population (As on 2016)
0,West Humber-Clairville,43.71516,-79.59251,33312
1,Mount Olive-Silverstone-Jamestown,43.74559,-79.58605,32954
2,Thistletown-Beaumond Heights,43.73755,-79.56456,10360
3,Rexdale-Kipling,43.72424,-79.56643,10529
4,Elms-Old Rexdale,43.71964,-79.55064,9456
...,...,...,...,...
135,West Hill,43.76436,-79.16839,27392
136,Woburn,42.47927,-71.15016,53485
137,Eglinton East,43.74264,-79.24017,22776
138,Scarborough Village,43.73626,-79.21600,16724


In [7]:
# Creating a csv file as the server is not accessible for all time

Toronto_Data.to_csv('toronto_latlong_data.csv', index=False)

In [7]:
# Fetching the latitude and Longitude data from the stored CSV file.

Toronto_Data = pd.read_csv("./toronto_latlong_data.csv")
Toronto_Data.head()

,Neighbourhood,Latitude,Longitude,Population (As on 2016)
0,West Humber-Clairville,43.71516,-79.59251,33312
1,Mount Olive-Silverstone-Jamestown,43.74559,-79.58605,32954
2,Thistletown-Beaumond Heights,43.73755,-79.56456,10360
3,Rexdale-Kipling,43.72424,-79.56643,10529
4,Elms-Old Rexdale,43.71964,-79.55064,9456


In [4]:
CLIENT_ID = ' ' # your Foursquare ID
CLIENT_SECRET = ' ' # your Foursquare Secret
ACCESS_TOKEN = ' ' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N51TMZ2YCXKTCIJKSRPTLUZOJLAH5CK3CGQPSJEOZEMPK1NQ
CLIENT_SECRET:TGKKJLJ0NHVKO0DL1HRKQJX5W53CZIFOO1EOZEBDFJ0R304W


Creating a function that will explore the neighbourhoods and fetch the venue details.

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['id'], # Fetches venue ID
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue',
                  'Venue ID',          
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [3]:
# Getting the data
toronto_venues = getNearbyVenues(names=Toronto_Data['Neighbourhood'],
                                   latitudes=Toronto_Data['Latitude'],
                                   longitudes=Toronto_Data['Longitude']
                                  )

print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
# Creating a csv file as the server is not accessible for all time

toronto_venues.to_csv('toronto_latlong_data.csv', index=False)

In [8]:
#toronto_venues.to_csv('toronto_venues_data.csv', index=False)


toronto_venues = pd.read_csv("./toronto_venues_data.csv")
toronto_venues.head()
#print(type(toronto_venues))

,Neighbourhood,Latitude,Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,West Humber-Clairville,43.71516,-79.59251,Tim Hortons,4be452122457a593c40eaa15,43.714657,-79.593716,Coffee Shop
1,West Humber-Clairville,43.71516,-79.59251,Xawaash,561d7a54498ed5c87ce4159d,43.715786,-79.593053,Mediterranean Restaurant
2,West Humber-Clairville,43.71516,-79.59251,Staples Rexdale,4d6ec39e12196ea832fa548e,43.718539,-79.594570,Paper / Office Supplies Store
3,West Humber-Clairville,43.71516,-79.59251,Swiss Pick,4cc21af082388cfae8f25d35,43.716150,-79.593843,Swiss Restaurant
4,West Humber-Clairville,43.71516,-79.59251,Enterprise Rent-A-Car,4d764ad4f5e4b1f7b07e0187,43.715260,-79.589320,Rental Car Location


Filter the dataframe by venue category = coffee shop

In [9]:
toronto_venues_coffee_shop = toronto_venues[toronto_venues['Venue Category'].str.contains("Coffee Shop")].reset_index(drop=True)
print(toronto_venues_coffee_shop.head())
print(toronto_venues_coffee_shop.shape)
print(type(toronto_venues_coffee_shop))

                       Neighbourhood  Latitude  Longitude        Venue  \
0             West Humber-Clairville  43.71516  -79.59251  Tim Hortons   
1  Mount Olive-Silverstone-Jamestown  43.74559  -79.58605  Tim Hortons   
2       Thistletown-Beaumond Heights  43.73755  -79.56456  Tim Hortons   
3                Etobicoke West Mall  43.64490  -79.56946  Tim Hortons   
4         Islington-City Centre West  43.63488  -79.53052  Coffee time   

                   Venue ID  Venue Latitude  Venue Longitude Venue Category  
0  4be452122457a593c40eaa15       43.714657       -79.593716    Coffee Shop  
1  4cd9d00734bb8cfa6576babf       43.742015       -79.589690    Coffee Shop  
2  4d84b99a02eb5481040f34f5       43.737881       -79.566891    Coffee Shop  
3  50d4866ee4b0c1554d94fef7       43.644742       -79.567681    Coffee Shop  
4  4c92baf61221b1f7c64514e2       43.635366       -79.532533    Coffee Shop  
(122, 8)
<class 'pandas.core.frame.DataFrame'>


Fetch the average user rating of each venue from Foursquare.

In [29]:
avg_rating = pd.DataFrame(columns=["Venue ID", "Average Rating"])

for i in range(toronto_venues_coffee_shop.shape[0]):
    venue_id = toronto_venues_coffee_shop.at[i, "Venue ID"]
    #print(venue_id)
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

    result = requests.get(url).json()
    try:
        average_rating = result['response']['venue']['rating']
    except:
        average_rating = "NaN" 
        
    avg_rating = avg_rating.append({"Venue ID":venue_id, "Average Rating":average_rating}, ignore_index=True)
    
avg_rating.head()

,Venue ID,Average Rating
0,4be452122457a593c40eaa15,6.8
1,4cd9d00734bb8cfa6576babf,5.5
2,4d84b99a02eb5481040f34f5,6.2
3,50d4866ee4b0c1554d94fef7,7.4
4,4c92baf61221b1f7c64514e2,NaN


In [30]:
#Store the dataframe into .csv file for future reference.
avg_rating.to_csv('toronto_venues_average_rating.csv', index=False)


,Venue ID,Average Rating
0,4be452122457a593c40eaa15,6.8
1,4cd9d00734bb8cfa6576babf,5.5
2,4d84b99a02eb5481040f34f5,6.2
3,50d4866ee4b0c1554d94fef7,7.4
4,4c92baf61221b1f7c64514e2,NaN


In [10]:
# Getting the data from the .csv file.
toronto_venues_average_rating = pd.read_csv("./toronto_venues_average_rating.csv")
toronto_venues_average_rating.head()

,Venue ID,Average Rating
0,4be452122457a593c40eaa15,6.8
1,4cd9d00734bb8cfa6576babf,5.5
2,4d84b99a02eb5481040f34f5,6.2
3,50d4866ee4b0c1554d94fef7,7.4
4,4c92baf61221b1f7c64514e2,NaN


Append the average rating column from toronto_venues_average_rating to the toronto_venues_coffee_shop dataframe.

In [11]:
toronto_venues_coffee_shop['Average Rating'] = toronto_venues_average_rating['Average Rating']
print(toronto_venues_coffee_shop.shape)
#toronto_venues_coffee_shop.head()


(122, 9)


In [12]:
# Dropping the columns not required.

final_dataset1 = toronto_venues_coffee_shop.drop(["Latitude","Longitude","Venue ID","Venue Latitude","Venue Longitude","Venue Category","Venue"],axis = "columns", inplace = False)
#final_dataset2 = final_dataset1.dropna()
final_dataset1


,Neighbourhood,Average Rating
0,West Humber-Clairville,6.8
1,Mount Olive-Silverstone-Jamestown,5.5
2,Thistletown-Beaumond Heights,6.2
3,Etobicoke West Mall,7.4
4,Islington-City Centre West,NaN
...,...,...
117,Ionview,NaN
118,Malvern,6.9
119,Eglinton East,6.9
120,Scarborough Village,NaN


In [13]:
# Dropping the rows with NaN values.

final_dataset1.dropna(inplace=True)
final_dataset1.reset_index(drop=True,inplace=True)
final_dataset1

,Neighbourhood,Average Rating
0,West Humber-Clairville,6.8
1,Mount Olive-Silverstone-Jamestown,5.5
2,Thistletown-Beaumond Heights,6.2
3,Etobicoke West Mall,7.4
4,New Toronto,6.0
...,...,...
104,Kennedy Park,6.0
105,Ionview,6.3
106,Malvern,6.9
107,Eglinton East,6.9


In [14]:
# Sorting the dataset in ascending order for calculation purpose.
final_dataset1.sort_values(by=['Neighbourhood'], inplace=True)
final_dataset1.reset_index(drop=True,inplace=True)
final_dataset1

,Neighbourhood,Average Rating
0,Alderwood,5.8
1,Annex,6.3
2,Bay Street Corridor,8.5
3,Bay Street Corridor,8.2
4,Bay Street Corridor,9.1
...,...,...
104,Yonge-St.Clair,6.1
105,Yonge-St.Clair,6.2
106,York University Heights,7.2
107,Yorkdale-Glen Park,6.3


Counting the no of coffee shop in each neighbourhood and their mean average rating.

In [15]:
final_dataset2 = pd.DataFrame(columns=["Neighbourhood", "No of Coffee Shop", "Average Rating"])
count = 0
c=0
for i in range(final_dataset1.shape[0]):
    c=c+count
    
    
    if(c<final_dataset1.shape[0]):
        value1 = final_dataset1.at[c,"Neighbourhood"]
    else:
        break
        
       
    count=0
    
    avg_rating = 0
    
    for j in range(final_dataset1.shape[0]):
        
        if (value1==final_dataset1.at[j,"Neighbourhood"]):
            count=count+1
            avg_rating = avg_rating + final_dataset1.at[j,"Average Rating"] 
    avg_rating_final = avg_rating / count
    
    final_dataset2 = final_dataset2.append({"Neighbourhood":value1, "No of Coffee Shop":count, "Average Rating":avg_rating_final}, ignore_index=True)
    
    
    
    
final_dataset2.head()

,Neighbourhood,No of Coffee Shop,Average Rating
0,Alderwood,1,5.8000
1,Annex,1,6.3000
2,Bay Street Corridor,8,7.7875
3,Bedford Park-Nortown,1,7.0000
4,Briar Hill-Belgravia,1,7.3000


In [16]:
final_dataset2.shape

(49, 3)

In [17]:
df.head()

,HoodID,Neighbourhood,Pop2016,Pop2011,PopChg11t16
0,1,West Humber-Clairville,33312,34100,-0.023109
1,2,Mount Olive-Silverstone-Jamestown,32954,32788,0.005063
2,3,Thistletown-Beaumond Heights,10360,10138,0.021898
3,4,Rexdale-Kipling,10529,10488,0.003909
4,5,Elms-Old Rexdale,9456,9550,-0.009843


Getting the population data in the final dataset.

In [18]:
final_dataset3 = pd.DataFrame(columns=["Neighbourhood", "No of Coffee Shop", "Average Rating","Population"])

for i in range(final_dataset2.shape[0]):
    val1 = final_dataset2.at[i,"Neighbourhood"]
    val2 = final_dataset2.at[i,"No of Coffee Shop"]
    val3 = final_dataset2.at[i,"Average Rating"]
    for j in range(df.shape[0]):
        val4 = df.at[j,"Neighbourhood"]
        if val1 == val4:
            val5 = df.at[j,"Pop2016"]
            break
    final_dataset3 = final_dataset3.append({"Neighbourhood":val1,"No of Coffee Shop":val2, "Average Rating":val3,"Population":val5}, ignore_index=True)
    
print(final_dataset3.shape)
final_dataset3.head()

(49, 4)


,Neighbourhood,No of Coffee Shop,Average Rating,Population
0,Alderwood,1,5.8000,12054
1,Annex,1,6.3000,30526
2,Bay Street Corridor,8,7.7875,25797
3,Bedford Park-Nortown,1,7.0000,23236
4,Briar Hill-Belgravia,1,7.3000,14257


In [19]:
# Calculating population / coffee shop and adding it into a column.
final_dataset3 ["Population / Coffee Shop"] = (final_dataset3['Population'] / final_dataset3['No of Coffee Shop'])
final_dataset3.head()

,Neighbourhood,No of Coffee Shop,Average Rating,Population,Population / Coffee Shop
0,Alderwood,1,5.8000,12054,12054
1,Annex,1,6.3000,30526,30526
2,Bay Street Corridor,8,7.7875,25797,3224.62
3,Bedford Park-Nortown,1,7.0000,23236,23236
4,Briar Hill-Belgravia,1,7.3000,14257,14257


In [20]:
# Sorting the dataset into descending order.
final_dataset3.sort_values(by=['Population / Coffee Shop'], inplace=True, ascending=False)
final_dataset3.reset_index(drop=True,inplace=True)
final_dataset3.head()

,Neighbourhood,No of Coffee Shop,Average Rating,Population,Population / Coffee Shop
0,Malvern,1,6.9,43794,43794
1,West Humber-Clairville,1,6.8,33312,33312
2,Mount Olive-Silverstone-Jamestown,1,5.5,32954,32954
3,Annex,1,6.3,30526,30526
4,York University Heights,1,7.2,27593,27593


In [21]:
# Scaling the Population / Coffee Shop column in scale of 0 - 10.

final_dataset3 ["Pop/CofShop Scaled"] = (final_dataset3["Population / Coffee Shop"]/(final_dataset3["Population / Coffee Shop"].max())*10)
final_dataset3.head()

,Neighbourhood,No of Coffee Shop,Average Rating,Population,Population / Coffee Shop,Pop/CofShop Scaled
0,Malvern,1,6.9,43794,43794,10
1,West Humber-Clairville,1,6.8,33312,33312,7.60652
2,Mount Olive-Silverstone-Jamestown,1,5.5,32954,32954,7.52478
3,Annex,1,6.3,30526,30526,6.97036
4,York University Heights,1,7.2,27593,27593,6.30063


In [22]:
# calculating the business opportunity index from the resultant dataset.

final_dataset3 ["Business Opportunity Index"] = (final_dataset3['Average Rating'] / final_dataset3['Pop/CofShop Scaled'])
final_dataset3.sort_values(by=['Business Opportunity Index'], inplace=True)
final_dataset3.head()

,Neighbourhood,No of Coffee Shop,Average Rating,Population,Population / Coffee Shop,Pop/CofShop Scaled,Business Opportunity Index
0,Malvern,1,6.9,43794,43794,10,0.69
2,Mount Olive-Silverstone-Jamestown,1,5.5,32954,32954,7.52478,0.730919
1,West Humber-Clairville,1,6.8,33312,33312,7.60652,0.89397
3,Annex,1,6.3,30526,30526,6.97036,0.903827
4,York University Heights,1,7.2,27593,27593,6.30063,1.14274
